This script generates the data used by the stand-alone activity recognition dashboard

In [ ]:
import datetime
import json
import os

import tator
import progressbar

In [ ]:
#
# Editable section
#
host = "https://www.tatorapp.com"
token = ""
tator_api = tator.get_api(host=host, token=token)

# Tator specific constants
project = 31
state_type = 149
multi_type = 55

# Define the vessels
vessels = {}
vessels["Arctic Fury (AF)"] = {"tag": "AF-"}
vessels["Lisa Melinda (LM)"] = {"tag": "LM-"}
vessels["Noahs Ark (NA)"] = {"tag": "NA-"}

vessel_tag_name_map = {}
for name in vessels:
    vessel_tag_name_map[vessels[name]["tag"]] = name

# Define the colors used on the timeline
color_map = {}
color_map["Background"] = "#6E5CA3"
color_map["Fish Activity"] = "#915CA3"
color_map["Fish Presence"] = "#5C6EA3"
color_map["Net Activity"] = "#6E5CA3"
color_map["Offloading"] = "#915CA3"

In [ ]:
#
# Gather all the sections and split them up by vessel
#
for vessel_name in vessels:
    this_vessel = vessels[vessel_name]
    this_vessel["sections"] = []

sections = tator_api.get_section_list(project=project)
for section in sections:
    if "_Trip" in section.name:
        
        found_match = False
        for tag in vessel_tag_name_map:
            vessel_name = vessel_tag_name_map[tag]
            if tag in section.name:
                vessels[vessel_name]["sections"].append(section)
                found_match = True
                break
                
        if not found_match:
            print(f"Did not find matching vessel for {section.name}")
            
for name in vessels:
    print(f"{name} - {len(vessels[name]['sections'])} trips")

In [ ]:
#
# For each of the vessels, grab the states and medias per section
#
def filename_to_timestamp(filename):
    """
    Turns a filename into a datetime object. Assumes `filename` is of the format
    `<extra>_<%Y%m%d>_<%H%M%S>.<extension>`
    """
    return datetime.datetime.strptime(
        "_".join(os.path.splitext(filename)[0].split("_")[1:]), "%Y%m%d_%H%M%S"
    )

for vessel_name in vessels:
    
    fps = None
    this_vessel = vessels[vessel_name]
    this_vessel["multis"] = []
    this_vessel["states"] = []
    this_vessel["valid_sections"] = []
    this_vessel["fps"] = [] # Assume the FPS is the same for all videos in the multi
    this_vessel["media_start_time_map"] = []
    this_vessel["global_start_time"] = []
    this_vessel["global_max_frame"] = []
    
    bar = progressbar.ProgressBar()
    for section in bar(this_vessel["sections"]):
        try:
            multis = tator_api.get_media_list(project=project, type=multi_type, section=section.id)

            prime_media_ids = [multi.media_files.ids[0] for multi in multis]
            states = tator_api.get_state_list(project=project, type=state_type, media_id=prime_media_ids)
            
            prime_medias = tator_api.get_media_list_by_id(project, {"ids": prime_media_ids})
            prime_media_map = {}
            for media in prime_medias:
                prime_media_map[media.id] = media
            
            global_start_time = datetime.datetime.now()
            media_start_time_map = {}
            for multi in multis:
                start_time = filename_to_timestamp(multi.name)
                for media_id in multi.media_files.ids:
                    media_start_time_map[media_id] = start_time
                
                if start_time < global_start_time:
                    global_start_time = start_time
                
            global_max_frame = 0
            for multi in multis:
                start_time = filename_to_timestamp(multi.name)
                max_frame = (start_time - global_start_time).total_seconds() * 15 + prime_media_map[multi.media_files.ids[0]].num_frames
                
                if max_frame > global_max_frame:
                    global_max_frame = max_frame
                
            this_vessel["fps"].append(15)
            this_vessel["multis"].append(multis)
            this_vessel["states"].append(states)
            this_vessel["media_start_time_map"].append(media_start_time_map)
            this_vessel["valid_sections"].append(section)
            this_vessel["global_start_time"].append(global_start_time)
            this_vessel["global_max_frame"].append(global_max_frame)
        except Exception as exc:
            print(exc)
            print(f"Error with {section.name}")
        

In [ ]:
#
# Log the activity, start frame, end frame. Convert frames to time.
# Stitch all the videos together and apply global frames/time to each event.
#
vessel_json_data = []

for vessel_name in vessels:
    this_vessel = vessels[vessel_name]
    this_vessel["activities"] = []
    num_trips = len(this_vessel["valid_sections"])
    
    trip_json_data = []
    for trip_index in range(num_trips):
        
        section = this_vessel["valid_sections"][trip_index]
        trip_name = section.name
        global_start_time = this_vessel["global_start_time"][trip_index]
        global_max_frame = this_vessel["global_max_frame"][trip_index]
        
        activities = {
            "Fish Presence": {},
            "Fish Activity": {},
            "Net Activity": {},
            "Offloading": {},
            "Background": {}}
        for key in activities:
            activities[key] = {
                "start_state_id": [],
                "end_state_id": [],
                "global_start_frame": [],
                "start_frame": [],
                "start_time": [],
                "start_multi_media_id": [],
                "start_multi_media_link": [],
                "global_end_frame": [],
                "end_frame": [],
                "end_time": [],
                "end_multi_media_id": [],
                "end_multi_media_link": []
            }
        
        fps =  this_vessel["fps"][trip_index]
        states = this_vessel["states"][trip_index]
        media_start_time_map = this_vessel["media_start_time_map"][trip_index] 
        
        multis = this_vessel["multis"][trip_index]
        multi_map = {}
        for multi in multis:
            for media_id in multi.media_files.ids:
                multi_map[media_id] = multi.id
        
        # Collect all the activities and set the times
        for state in states:
            atype = state.attributes["Activity Type"]
            start_frame = state.attributes["Start Frame"]
            start_time = datetime.timedelta(seconds=(start_frame / fps)) + media_start_time_map[state.media[0]]
            end_frame = state.attributes["End Frame"]
            end_time = datetime.timedelta(seconds=(end_frame / fps)) + media_start_time_map[state.media[0]]
            multi_id = multi_map[state.media[0]]
            global_start_frame = (start_time - global_start_time).total_seconds() * fps
            
            start_multi_media_link = f"https://www.tatorapp.com/{project}/annotation/{multi_id}?section={section.id}&frame={start_frame}&selected_entity={state.id}&selected_type=state_{state.meta}&version={state.version}"
            end_multi_media_link = f"https://www.tatorapp.com/{project}/annotation/{multi_id}?section={section.id}&frame={end_frame}&selected_entity={state.id}&selected_type=state_{state.meta}&version={state.version}"
            
            activities[atype]["start_state_id"].append(state.id)
            activities[atype]["global_start_frame"].append(global_start_frame)
            activities[atype]["start_frame"].append(start_frame)
            activities[atype]["start_time"].append(start_time)
            activities[atype]["start_multi_media_id"].append(multi_id)
            activities[atype]["start_multi_media_link"].append(start_multi_media_link)
        
            activities[atype]["end_state_id"].append(state.id)
            activities[atype]["global_end_frame"].append(global_start_frame + (end_frame - start_frame))
            activities[atype]["end_frame"].append(end_frame)
            activities[atype]["end_time"].append(end_time)
            activities[atype]["end_multi_media_id"].append(multi_id)
            activities[atype]["end_multi_media_link"].append(end_multi_media_link)
            
        # Go through each of the activities and merge gaps if they are within a certain tolerance
        tolerance_seconds = 5
        final = {}
        for atype in activities:

            # Sort the lists based on start time
            unsorted_list = activities[atype]["start_time"]
            sort_indexes = sorted(range(len(unsorted_list)),key=unsorted_list.__getitem__)

            final[atype] = {}
            for key in activities[atype]:
                activities[atype][key] = [activities[atype][key][idx] for idx in sort_indexes]
                final[atype][key] = []
                
            # Now, decide which indexes to merge
            for idx in range(len(unsorted_list)):
                if idx > 0:
                    prev_end_time = final[atype]["end_time"][-1]
                    curr_start_time = activities[atype]["start_time"][idx]
                    delta_time = (curr_start_time - prev_end_time).total_seconds()
                    
                    if delta_time <= tolerance_seconds:
                        final[atype]["global_end_frame"][-1] = activities[atype]["global_end_frame"][idx]
                        final[atype]["end_frame"][-1] = activities[atype]["end_frame"][idx]
                        final[atype]["end_time"][-1] = activities[atype]["end_time"][idx]
                        final[atype]["end_multi_media_id"][-1] = activities[atype]["end_multi_media_id"][idx]
                        final[atype]["end_multi_media_link"][-1] = activities[atype]["end_multi_media_link"][idx]
                    else:
                        for key in activities[atype]:
                            final[atype][key].append(activities[atype][key][idx])
                else:
                    for key in activities[atype]:
                        final[atype][key].append(activities[atype][key][idx])
            
        this_vessel["activities"].append(final)
        
        # Create the JSON object for the activities
        truth_data_json = []
        for atype in sorted(final.keys()):
            truth_data_json.append({
                "activity": atype,
                "data": final[atype],
                "color": color_map[atype],
            })
            for start_time in truth_data_json[-1]["data"]["start_time"]:
                truth_data_json[-1]["data"]["start_time"] = str(start_time)
            for end_time in truth_data_json[-1]["data"]["end_time"]:
                truth_data_json[-1]["data"]["end_time"] = str(end_time)
                
    
        # Create the JSON object for the trip
        trip_json_data.append({
            "tripName": trip_name,
            "truthData": truth_data_json,
            "globalMaxFrame": global_max_frame,
        })
    
    vessel_json_data.append({
        "vesselName": vessel_name,
        "trips":  trip_json_data
    })

In [ ]:
#
# Create the JS file the dashboard will use directly
#
with open("ar_dashboard_data.js", "w") as file_handle:
    file_handle.write(f"const dataDate = \"{datetime.datetime.now()}\";")
    file_handle.write(f"const vesselData = {json.dumps(vessel_json_data)};")